In [ ]:
# Irregular Tokens. Tokenize the dataset using your own tokenizer. Identify the top-20 most frequent
# tokens which are NOT standard English words (including their morphological forms).

In [ ]:
import enchant
import pandas as pd
import os.path as path
import ast
import operator
import re
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import nltk
import string
from nltk import pos_tag, word_tokenize
import argparse
import os

In [ ]:
def tokenize_code(code):
    code_lines = list(filter(None, code.split('\n')))
    for line in code_lines:
        # store line info then append to 'post' array
        line_info = [line]
        tokens = code_tokenizer.tokenize(line)
        # remove function definition from matching the case of function call
        fun_def = False
        for t in tokens:
            if re.match(function_call, t) and not re.match(r'(?:\w+\.\w+\s*\(.*\))',t):
                # if there is a { symbol after a function
                # re-tokenize the line
                if t != tokens[-1] and '{' in tokens[tokens.index(t)+1:]:
                    fun_def = True
                    break
        if fun_def:
            temp_patterns = patterns =  keywords + '|' + string_literal + '|' + comments + '|'\
                + directory + '|' + identifier + '|' + floating_literal + '|' \
                + hex_literal + '|' + operators + '|' + octal_literal + '|' \
                + decimal_literal
            temp_code_tokenizer = RegexpTokenizer(temp_patterns)
            tokens = temp_code_tokenizer.tokenize(line)
        line_info.append(tokens)

        if tokens and re.match(comments, tokens[-1]):
            t = tokens[-1]
            t = t.replace('//', '')
            # tokenize comments as natural language
            t = tokenize_natural_language(t)
            last = line_info.pop()
            last.pop()
            line_info.append(last + ['//'] + t)




def split_punc(token):

    punc = string.punctuation.replace('/', '')
    punctuation_mark = r"[{}]".format(punc) # create the pattern
    left = '<{[('
    right = '>}])'
    if re.match(punctuation_mark, token[0]) and not re.match(punctuation_mark, token[-1]):
#         print("first char is punctuation mark")
        token_lst = []
        iter_token = iter(range(len(token)))
        for i in iter_token: 
            if token[i] in punc:
                if token[i] in left:
                    corresponding_right = right[left.index(token[i])]
                    if corresponding_right in token:
                        token_lst.append(token[i:])
                        token_lst.insert(0, token[i:])
                        break
                token_lst.append(token[i])
            else:
                token_lst.append(token[i:])
                # copy the word token in front for later processing
                token_lst.insert(0, token[i:])
                break
        omit = ''
        lst = token_lst[:]
        for i in range(len(lst)-1, -1, -1):
            if token_lst[i] == '.':
                omit += token_lst[i]
            else:
                if len(omit) >= 3:
                    token_lst = token_lst[:i+1]
                    token_lst.append(omit)
                    if (i+len(omit)+1) < len(lst):
                        token_lst += lst[i+len(omit)+1:]
                    break
        return token_lst
    elif re.match(punctuation_mark, token[-1]) and not re.match(punctuation_mark, token[0]):
#         print("last char is punctuation mark")
        token_lst = []
        iter_token = iter(range(len(token)-1, -1, -1))
        for i in iter_token:
            if token[i] in punc:
                
                if token[i] in right:
                    corresponding_left = left[right.index(token[i])]
                    if corresponding_left in token:
                        token_lst.insert(0, token[:i+1])
                        token_lst.insert(0, token[:i+1])
                        break
                token_lst.insert(0, token[i])
            else:
                token_lst.insert(0, token[:i+1])
                # copy the word token in front for later processing
                token_lst.insert(0, token[:i+1])
                break
        omit = ''
        lst = token_lst[:]
        for i in range(len(lst)-1, -1, -1):
            if token_lst[i] == '.':
                omit += token_lst[i]
            else:
                if len(omit) >= 3:
                    token_lst = token_lst[:i+1]
                    token_lst.append(omit)
                    if (i+len(omit)+1) < len(lst):
                        token_lst += lst[i+len(omit)+1:]
                    break        
        return token_lst
    else:
        return [token, token]

def tokenize_natural_language(txt):
#     re_entity = r'(?:[A-Z][A-Za-z]*\s)*[A-Z][A-Za-z]*'

#     entities = RegexpTokenizer(re_entity).tokenize(txt)
#     name_entities = []
#     for entity in entities:
#         if " " not in entity:
#             break
#         satisfier = True
#         pos = nltk.pos_tag(entity.split(" "))[0][1]:
#         if pos == 'DT' or pos == 'PRP'or pos == 'CC' or pos == 'WRB' or pos == 'IN':
#             satisfier = False
#             break
#         if satisfier:
#             name_entities.append(entity)

#     if name_entities:
#         print(name_entities)
#     dic = {entity:entity.split(" ") for entity in name_entities}
#     entity_tokens = [item for sublist in dic.values() for item in sublist]
    space_tokenizer = RegexpTokenizer(r'\S+')
    tokens = space_tokenizer.tokenize(txt)

    new_token_list = []
#     find_entity = False
    iter_tokens = iter(tokens)
    for token in iter_tokens:
#         for key, val in dic.items():
#             index = tokens.index(token)
#             if token in val and (index+len(val)) < len(tokens):
#                 reconstruct = " ".join([tokens[t] for t in range(index, index + len(val))])
#                 splited = split_punc(reconstruct)
#                 if splited[0] == key:
#                     new_token_list += splited[1:]
#                     if len(val) != 1:
#                         for n in range(len(val)-1):
#                             next(iter_tokens)
#                     find_entity = True
#                     break
#         if find_entity:
#             find_entity = False
#             continue

        new_token_list += split_punc(token)[1:]

    # split abbreviation
    abbreviation = r'(?:n\'t$)|(?:\'s$)|(?:\'m$)|(?:\'ve$)|(?:\'re$)|(?:\'ll$)|(?:\'d$)'
    old_tokens = new_token_list[:]
    match_count = 0
    for i, t in enumerate(old_tokens):
        if re.search(abbreviation, t):
            m = re.search(abbreviation, t)
            word_root = t[:m.span()[0]]
            suffix = t[m.span()[0]:m.span()[1]]
            new_token_list[i+match_count] = word_root
            new_token_list.insert(i+match_count+1, suffix)
            match_count += 1

    return new_token_list

def differentiate_code_nl(p_id, body, code_df):
    tokenization=[]
    count = 0

    no_match = False
    # dic is used to store the overall info for one post
    dic = {'text' : body, 'tokens' : []}
    # separate code and natural language, store into parts
    parts = []
    # 1 for code, 0 for natural language
    code_or_not = []
    # if the body does not have any code block, no need to process
    if p_id in code_df.index:
        code_blks = code_df.loc[p_id]['Code']
        # check code blk position in a post
        # arrange them in order
        position = {}
        if isinstance(code_blks, str):
            if body.find(code_blks) == -1:
                no_match = True
            elif len(code_blks) > 20:
                position[body.index(code_blks)] = code_blks
        else:
            for code_blk in code_blks.values:
                # Note: the 'no-match' problem is due to different formatting/parsing methods from xml to csv
                if not isinstance(code_blk, str):
                    continue
                if body.find(code_blk) == -1:
                    no_match = True
                    break
                if len(code_blk) > 20:
                    position[body.index(code_blk)] = code_blk
        if no_match:
            return None
        keylist = sorted(position.keys())
        for key in keylist:
            code = position[key]
            # handle the situation where the code appears more than once
            # split based on fist occurence
            rest = body.split(code, 1)
            parts += ([rest[0], code])
            code_or_not += ([0, 1] if rest[0] else [1])
            if len(rest) > 2:
                parts.append(rest[-1])
                if rest[-1].strip():
                    code_or_not.append(0)

            parts = list(filter(None, parts))
            body = rest[-1]
        if keylist and len(rest) == 2 and rest[1].strip():
            parts.append(rest[1])
            code_or_not.append(0)
    # if there is not code block
    if not parts:
        parts.append(body)
        code_or_not=[0]
    # post is used to store line by line info
    post = []
    natural_part = []
    code_part = []
    for index, value in enumerate(parts):
        if code_or_not[index]:
#             dic, post = tokenize_code(value, dic, post)
            code_part.append(value)
        else:
#             token = tokenize_natural_language(value)
#             post.append([value, token])
#             dic['tokens'] += token
            natural_part.append(value)
    # df for each post, do line by line annotation

#     tokenization.append([p_id, dic['text'], dic['tokens']])
#     count +=1
    
    # df for all posts
#     results = pd.DataFrame(tokenization, columns = ['post_id', 'text', 'tokens'])

    return natural_part, code_part

In [ ]:
def sentence_has_irregular_token(sentence, irregular_token):
    for token in sentence:
        if token in irregular_token:
            return True
    return False

In [ ]:
english_dictionary = enchant.Dict("en_US")

In [ ]:
token_frequency_dic = {} # [token_content] : frequency
df = pd.DataFrame.from_csv('../processed_data/tokenizer_result/all/overall_results.csv')
generated_tokens = [ast.literal_eval(t) for t in df.tokens.tolist()]

for sentence in generated_tokens:
    for token in sentence:
        if token not in token_frequency_dic.keys():
            token_frequency_dic[token] = 1
        else:
            token_frequency_dic[token] += 1

In [ ]:
top_20 = {}
floating_literal = r'''^\d+\.\d*(?:[eE][+-]\d+)?i?     
                | \d+[eE][+-]\d+i?               
                | \.\d+(?:[eE][+-]\d+)?i?$     
                '''
decimal_literal = r'^\d+$'
operators = r'^[%/\+\-\*\,;\$><!:\.\|&\^=\(\)\[\]\{\}\"\'\#\?_]+$'

for token, frequency in sorted(token_frequency_dic.items(), key=operator.itemgetter(1), reverse=True):
    if english_dictionary.check(token):
        continue
    if re.match(decimal_literal, token):
        # the token is an number
        continue
    if re.match(floating_literal, token):
        # the token is an float number
        continue
    if re.match(operators, token):
        # the token is an number
        continue
        
    if token in ["'d","'m", "'v", "'re", "'ve", "'s", "n't"]:
        # special english token
        continue
    top_20[token] = frequency
    if (len(top_20.keys()) == 20):
        break
#     print(token)
#     print(frequency)

In [ ]:
write = "token:\tfrequency\n"
for token, frequency in sorted(top_20.items(), key=operator.itemgetter(1), reverse=True):
    write += str(token) + ':\t' + str(frequency) + '\n'
    
# print(write)
with open('../Stats/top_20_irregular.txt', 'w') as f:
    f.write(write)

In [ ]:
# POS Tagging. Randomly select 10 sentences from the dataset where each sentence contains at
# least one irregular token. Apply POS tagging on the sentences based on your own tokenization
# results. Show and discuss the tagging results.

In [ ]:
ALL_codes = pd.read_csv("../processed_data/code.csv")
ALL_codes.set_index('PostId', inplace=True)

In [ ]:
sentence_stop = ['\.', '\?','!','\\n','\.\.\.',';']
code_stop = ['\\n']

In [ ]:
def select_10_sentence(df):
    selected_10_tokens = []
    selected_10_sentences = []
    selected_id = []
    id_list = list(range(len(df)))
    np.random.shuffle(id_list)
    for l in id_list:
        post_text = df.iloc[l]['text']
        nature, code = differentiate_code_nl(l,post_text, ALL_codes)

        for nature_part in nature:
            stops = []
            for punc in sentence_stop:
                stops += [m.start() for m in re.finditer(punc, nature_part)]

            begin = 0
            for i in sorted(stops):
                sentence = nature_part[begin:i+1]
                begin = i+1

                sentence_token = tokenize_natural_language(sentence)
                if sentence_has_irregular_token(sentence_token, top_20.keys()):
                    selected_10_tokens.append(sentence_token)
                    selected_id.append(df.iloc[l]['post_id'])
                    selected_10_sentences.append(sentence)
                if (len(selected_10_sentences) >= 10):
                    return selected_10_tokens,selected_10_sentences, selected_id
                
        for code_part in code:
            stops = []
            for punc in code_stop:
                stops += [m.start() for m in re.finditer(punc, code_part)]

            begin = 0
            for i in sorted(stops):
                sentence = code_part[begin:i+1]
                begin = i+1
                sentence_token = tokenize_code(sentence)
                if sentence_has_irregular_token(sentence_token, top_20.keys()):
                    if sentence in selected_10_sentences:
                        continue
                    selected_10_tokens.append(sentence_token)
                    selected_id.append(df.iloc[l]['post_id'])
                    selected_10_sentences.append(sentence)
                if (len(selected_10_sentences) >= 10):
                    return selected_10_tokens,selected_10_sentences, selected_id
                
    return selected_10_tokens,selected_10_sentences, selected_id
            


In [ ]:
res_tokens, res_questions, res_id = select_10_sentence(df)

In [ ]:
import nltk
with open('../Stats/postagging_10_sentences_result.txt', 'w') as f:
    f.write("Sentence Selected:\n")
    count = 1
    for sentence in res_questions:
        f.write(str(count) + ". " + sentence)
        if sentence[-1] != '\n':
            f.write('\n')
        count += 1
    f.write("\nRunning Result:\n")
    count = 1
    for token in res_tokens:
        f.write(str(count) + '.')
        for i in nltk.pos_tag(token):
            f.write("\t" + str(i) + "\n")
        count += 1
        f.write("\n")
    
print(open('../Stats/postagging_10_sentences_result.txt', 'r').read())